# <h1 align="center"><font color="gree">Synthetic Data Generation</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Links de estudo:

* [LangChain: Generate Synthetic Data](https://python.langchain.com/docs/tutorials/data_generation/)

* [sudarshan-koirala](https://github.com/sudarshan-koirala/youtube-stuffs/blob/main/langchain/synthetic_data_generation.ipynb)

Uma vez que os dados sintéticos são gerados, precisamos avaliá-los para garantir que estejam OK para uso em tarefas posteriores. Existem muitas bibliotecas e sites que oferecem esse tipo de solução. Mas aqui, focamos na parte `GenAI`.


# <font color="red">Use case</font>

Dados sintéticos referem-se a dados gerados artificialmente que imitam as características de dados reais sem conter nenhuma informação de indivíduos ou entidades reais. Normalmente, são criados por meio de modelos matemáticos, algoritmos ou outras técnicas de geração de dados. Dados sintéticos podem ser usados ​​para uma variedade de propósitos, incluindo testes, pesquisas e treinamento de modelos de aprendizado de máquina, preservando a privacidade e a segurança.

Benefícios dos Dados Sintéticos:

* `Privacidade e segurança:` Nenhum dado pessoal real corre risco de violações.

* `Aumento de dados (Data Augmentation):` Expande conjuntos de dados para aprendizado de máquina.

* `Flexibilidade:` Cria cenários específicos ou raros.

* `Custo-benefício:` Geralmente mais barato do que a coleta de dados do mundo real.

* `Conformidade regulatória:` Ajuda a navegar em leis rígidas de proteção de dados.

* `Robustez do modelo:` Pode levar a modelos de IA de melhor generalização.

* `Prototipagem rápida:` Permite testes rápidos sem dados reais.

* `Experimentação controlada:` Simula condições específicas.

* `Acesso aos dados:` Alternativa quando dados reais não estão disponíveis.


<font color="pink">Observação: Apesar dos benefícios, dados sintéticos devem ser usados ​​com cuidado, pois nem sempre podem capturar complexidades do mundo real.</font>


<font colr="orange">Neste notebook, vamos nos aprofundar na geração de `registros de faturamento médico` sintéticos usando a `biblioteca langchain`. Esta ferramenta é particularmente útil quando você quer desenvolver ou testar algoritmos, mas não quer usar dados reais de pacientes devido a preocupações com privacidade ou problemas de disponibilidade de dados.</font>

# <font color="red">Setup</font>

* Primeiro, você precisará ter a `biblioteca langchain` instalada, junto com suas dependências. Como estamos usando a `cadeia de geradores OpenAI`, instalaremos isso também. Como esta é uma biblioteca experimental, precisaremos incluir `langchain_experimental` em nossas instalações.

* [Pydantic](https://docs.pydantic.dev/latest/): Biblioteca de validação de dados para Python

In [1]:
%%capture
#%pip install -U langchain langchain_experimental openai
%pip install --upgrade --quiet  langchain langchain_experimental langchain-openai

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key  = os.environ['OPENAI_API_KEY']
#Eddy_key_openai  = os.environ['OPENAI_API_KEY']
#from openai import OpenAI
#client = OpenAI(api_key=Eddy_key_openai)

In [3]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
#from langchain.chat_models import ChatOpenAI

#from langchain.pydantic_v1 import BaseModel
from pydantic import  BaseModel
from langchain_experimental.tabular_synthetic_data.base import SyntheticDataGenerator
from langchain_experimental.tabular_synthetic_data.openai import create_openai_data_generator, OPENAI_TEMPLATE
from langchain_experimental.tabular_synthetic_data.prompts import SYNTHETIC_FEW_SHOT_SUFFIX, SYNTHETIC_FEW_SHOT_PREFIX

# <font color="red">Define your Data Model</font>

* Cada conjunto de dados tem uma estrutura ou um "esquema".

* A classe `MedicalBilling` abaixo serve como nosso `esquema para os dados sintéticos`.

* Ao definir isso, estamos informando nosso gerador de dados sintéticos sobre a forma e a natureza dos dados que esperamos.

In [4]:
class MedicalBilling(BaseModel):
    patient_id: int
    patient_name: str
    diagnosis_code: str
    procedure_code: str
    total_charge: float
    insurance_claim_amount: float

# <font color="red">Sample Data</font>

`Para orientar o gerador de dados sintéticos`, é útil fornecer alguns exemplos do mundo real. Esses exemplos servem como uma `"semente"` (seed) - eles são representativos do tipo de dados que você deseja, e o gerador os usará para criar mais dados que pareçam semelhantes.

Aqui estão alguns registros fictícios de cobrança médica:

In [5]:
examples = [
    {"example": """Patient ID: 123456, Patient Name: Eddy Giusepe, Diagnosis Code: J20.9, Procedure Code: 99203, Total Charge: $500, Insurance Claim Amount: $350"""},
    {"example": """Patient ID: 789012, Patient Name: Ramos Gonzalo, Diagnosis Code: M54.5, Procedure Code: 99213, Total Charge: $150, Insurance Claim Amount: $120"""},
    {"example": """Patient ID: 345678, Patient Name: Dionisia, Diagnosis Code: E11.9, Procedure Code: 99214, Total Charge: $300, Insurance Claim Amount: $250"""},
]

# <font color="red">Craft a Prompt Template (`Elabore um Modelo de Prompt`)</font>

O `gerador` não sabe magicamente como criar nossos dados; precisamos guiá-lo. Fazemos isso criando um `Prompt Template`. Este modelo ajuda a `instruir o modelo de linguagem subjacente` sobre como produzir dados sintéticos no formato desejado.

In [6]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)


O `FewShotPromptTemplate` inclui:

* `prefix` e `suffix`: provavelmente contêm contexto ou instruções de orientação.

* `examples`: os dados de amostra que definimos anteriormente.

* `input_variables`: essas variáveis ​​(`"subject"`, `"extra"`) são marcadores (placeholders) de posição que você pode preencher dinamicamente mais tarde. `Por exemplo`, `"subject"` pode ser preenchido com `"medical_billing"` para orientar o modelo mais adiante.

* `example_prompt`: este modelo de prompt é o formato que queremos que cada linha de exemplo tenha em nosso prompt.

# <font color="red"> Creating the Data Generator</font>

Com o `esquema` e o `prompt` prontos, o próximo passo é criar o `gerador de dados`. Este objeto sabe como se comunicar com o modelo de linguagem subjacente para obter dados sintéticos.

In [7]:
from langchain_openai import ChatOpenAI

synthetic_data_generator = create_openai_data_generator(output_schema=MedicalBilling,
                                                        llm=ChatOpenAI(model="gpt-4o-mini", temperature=1),
                                                        prompt=prompt_template,
                                                    )

# <font color="red">Generate Synthetic Data</font>

Por fim, vamos obter nossos dados sintéticos!

In [8]:
synthetic_results = synthetic_data_generator.generate(subject="medical_billing",
                                                      extra="O nome deve ser escolhido aleatoriamente. Faça algo que você normalmente não escolheria.",
                                                      runs=5,
                                                     )

Este comando pede ao gerador para produzir `5` registros de faturamento médico sintético. Os resultados são armazenados em `synthetic_results`. A saída será uma lista dos modelos `pydantic MedicalBilling`.

In [9]:
type(synthetic_results)

list

# <font color="red">Visualize the Generated Synthetic Data</font>

In [10]:
len(synthetic_results)

5

In [11]:
synthetic_results

[MedicalBilling(patient_id=456789, patient_name='Miranda Chen', diagnosis_code='I10', procedure_code='99204', total_charge=750.0, insurance_claim_amount=600.0),
 MedicalBilling(patient_id=234567, patient_name='Xander Quill', diagnosis_code='G43.909', procedure_code='99215', total_charge=400.0, insurance_claim_amount=320.0),
 MedicalBilling(patient_id=987654, patient_name='Elowen Thorne', diagnosis_code='F32.9', procedure_code='99213', total_charge=500.0, insurance_claim_amount=400.0),
 MedicalBilling(patient_id=543216, patient_name='Octavia Prism', diagnosis_code='M54.5', procedure_code='99203', total_charge=920.0, insurance_claim_amount=740.0),
 MedicalBilling(patient_id=135790, patient_name='Felix Caterwaul', diagnosis_code='R51.9', procedure_code='99214', total_charge=680.0, insurance_claim_amount=550.0)]

# <font color="red">Converting the synthetic data into Pandas Dataframe</font>

In [13]:
import pandas as pd

# Create a list of dictionaries from the objects
synthetic_data = []
for item in synthetic_results:
    synthetic_data.append({
        'patient_id': item.patient_id,
        'patient_name': item.patient_name,
        'diagnosis_code': item.diagnosis_code,
        'procedure_code': item.procedure_code,
        'total_charge': item.total_charge,
        'insurance_claim_amount': item.insurance_claim_amount
    })

# Create a Pandas DataFrame from the list of dictionaries
synthetic_df = pd.DataFrame(synthetic_data)

# Display the DataFrame
print(type(synthetic_df))


<class 'pandas.core.frame.DataFrame'>


In [14]:
synthetic_df

,patient_id,patient_name,diagnosis_code,procedure_code,total_charge,insurance_claim_amount
0,456789,Miranda Chen,I10,99204,750.0,600.0
1,234567,Xander Quill,G43.909,99215,400.0,320.0
2,987654,Elowen Thorne,F32.9,99213,500.0,400.0
3,543216,Octavia Prism,M54.5,99203,920.0,740.0
4,135790,Felix Caterwaul,R51.9,99214,680.0,550.0


<font color="orange">Comece a explorar com base no seu caso de uso e use a mesma abordagem para dados sensíveis reais. `Mas tenha cuidado, pois os dados sintéticos podem não capturar as complexidades do mundo real.`</font>